In [8]:
import pandas as pd

In [ ]:
# data start is maybe 12/08/2024 run date
# data is till the 1/24/2025 calendar week and 1/23/2025 run date

# Assumptions:
# No values for saturday? Maybe they are combined with Sunday's values
# Assuming each week pax_build resets and the values are based on the day before

# pax_build is the daily increase/decrease in passenger bookings, from run_date resets each calendar_week
# there are negative values in pax_build which indicates cancelations

# code to help understand the data

# df["meta_product_code"].value_counts()
# df[df["meta_product_code"] == "7N CARIBBEAN"]["ship"].value_counts()

# temp = df.sort_values(by=["calendar_week", "run_date"])
# temp = temp[temp["meta_product_code"] == "7N CARIBBEAN"]
# temp = temp[temp["ship"] == "AL"]
# temp = temp[temp["calendar_week"] == "2025-01-03"]
# temp = temp[temp["run_date"] == "2024-12-31"]
# temp = temp[temp["sail_date"] == "2026-02-22 00:00:00+00:00	"]
# temp[:200]

In [10]:
df = pd.read_excel('Track_case_study_data.xlsx', sheet_name='WTD_data')

# data cleaning
df['run_date'] = pd.to_datetime(df['run_date'])
df['calendar_week'] = pd.to_datetime(df['calendar_week'])
df['sail_date'] = pd.to_datetime(df['sail_date'])
df['day_of_week'] = df['run_date'].dt.dayofweek

day_mapping = {
    0: 'Monday',
    1: 'Tuesday',
    2: 'Wednesday',
    3: 'Thursday',
    4: 'Friday',
    5: 'Saturday',
    6: 'Sunday'
}
df['day_of_week'] = df['day_of_week'].map(day_mapping)


# calculate daily_pax from pax_build
df = df.sort_values(by=['ship', 'sail_date', 'calendar_week', 'run_date'])
df['daily_pax'] = df.groupby(['ship', 'sail_date', 'calendar_week'])['pax_build'].diff()
mask = df.groupby(['ship', 'sail_date', 'calendar_week'])['run_date'].transform('min') == df['run_date']
df.loc[mask, 'daily_pax'] = df.loc[mask, 'pax_build']

# we aren't considering days with cancelations
df = df[df['daily_pax'] > 0]
df.head()


,run_date,calendar_week,ship,sail_date,sail_year,sail_month,meta_product_code,pax_build,day_of_week,daily_pax
4764,2024-12-08,2024-12-13,AD,2024-12-07 00:00:00+00:00,2024,12,7N CARIBBEAN,1.838204,Sunday,1.838204
60200,2024-12-12,2024-12-13,AD,2024-12-07 00:00:00+00:00,2024,12,7N CARIBBEAN,-39.796944,Thursday,3.778482
36310,2024-12-13,2024-12-13,AD,2024-12-07 00:00:00+00:00,2024,12,7N CARIBBEAN,-37.890169,Friday,1.906774
50317,2024-12-08,2024-12-13,AD,2024-12-13 00:00:00+00:00,2024,12,7N CARIBBEAN,25.660515,Sunday,25.660515
91531,2024-12-09,2024-12-13,AD,2024-12-13 00:00:00+00:00,2024,12,7N CARIBBEAN,39.796452,Monday,14.135937


In [15]:
temp = df.sort_values(by=["calendar_week", "run_date"])
temp = temp[temp["meta_product_code"] == "SHORT CARIBBEAN"]
# temp = temp[temp["ship"] == "AL"]
temp = temp[temp["day_of_week"] == "Friday"]
# temp = temp[temp["calendar_week"] == "2025-01-03"]
# temp = temp[temp["run_date"] == "2024-12-31"]
temp = temp[temp["sail_year"] == 2025]
temp = temp[temp["sail_month"] == 9]

temp.head()

,run_date,calendar_week,ship,sail_date,sail_year,sail_month,meta_product_code,pax_build,day_of_week,daily_pax
59565,2024-12-13,2024-12-13,EN,2025-09-04 00:00:00+00:00,2025,9,SHORT CARIBBEAN,23.587975,Friday,2.936219
45344,2024-12-13,2024-12-13,EN,2025-09-08 00:00:00+00:00,2025,9,SHORT CARIBBEAN,6.158073,Friday,1.595542
74542,2024-12-13,2024-12-13,EN,2025-09-22 00:00:00+00:00,2025,9,SHORT CARIBBEAN,11.135080,Friday,3.529226
50352,2024-12-13,2024-12-13,FR,2025-09-25 00:00:00+00:00,2025,9,SHORT CARIBBEAN,17.408448,Friday,3.911112
21652,2024-12-13,2024-12-13,MA,2025-09-11 00:00:00+00:00,2025,9,SHORT CARIBBEAN,73.631524,Friday,18.326463


In [12]:
weekly_by_day = df.groupby(['sail_year', 'sail_month', 'meta_product_code', 'calendar_week', 'day_of_week'])['daily_pax'].sum().reset_index()
weekly_totals = weekly_by_day.groupby(['sail_year', 'sail_month', 'meta_product_code', 'calendar_week'])['daily_pax'].sum().reset_index()
weekly_totals = weekly_totals.rename(columns={'daily_pax': 'total_weekly_pax'})

result = pd.merge(weekly_by_day, weekly_totals, on=['sail_year', 'sail_month', 'meta_product_code', 'calendar_week'])
result['pct'] = (result['daily_pax'] / result['total_weekly_pax']) * 100
final_pcts = result.groupby(['sail_year', 'sail_month', 'meta_product_code', 'day_of_week'])['pct'].median().reset_index()

pivot = final_pcts.pivot_table(
    index=['sail_year', 'sail_month', 'meta_product_code'],
    columns='day_of_week',
    values='pct'
)

day_order = ['Sunday', 'Monday', 'Tuesday', 'Wednesday', 'Thursday', 'Friday', 'Saturday']
pivot = pivot.reindex(columns=day_order, fill_value=0)

pivot['Weekend'] = pivot.get('Saturday', 0) + pivot.get('Sunday', 0)

# formatting
final = pivot[['Weekend', 'Monday', 'Tuesday', 'Wednesday', 'Thursday', 'Friday']]
final = final.reset_index()
final.columns.name = None
final = final.rename_axis(None, axis=1) 

# Normalizes results
cols_to_normalize = ['Weekend', 'Monday', 'Tuesday', 'Wednesday', 'Thursday', 'Friday']
numeric_data = final[cols_to_normalize]
row_sums = numeric_data.sum(axis=1)
normalized_data = numeric_data.div(row_sums, axis=0) * 100
final[cols_to_normalize] = normalized_data
numerical_final = final.copy()

# adds % to the results
for i in cols_to_normalize:
    final[i] = round(final[i]).astype(int).astype(str) + "%"

final.columns = ["Sail Year", "Sail Month", "Product", "Weekend", "Monday", "Tuesday", "Wednesday", "Thursday", "Friday"]
final

,Sail Year,Sail Month,Product,Weekend,Monday,Tuesday,Wednesday,Thursday,Friday
0,2024,12,7N CARIBBEAN,65%,6%,5%,5%,6%,14%
1,2024,12,SHORT CARIBBEAN,51%,8%,12%,6%,4%,19%
2,2025,1,7N CARIBBEAN,48%,10%,10%,9%,9%,14%
3,2025,1,SHORT CARIBBEAN,37%,16%,9%,11%,12%,15%
4,2025,2,7N CARIBBEAN,37%,15%,12%,10%,12%,14%
...,...,...,...,...,...,...,...,...,...
69,2026,12,7N CARIBBEAN,8%,5%,3%,2%,4%,78%
70,2027,1,7N CARIBBEAN,16%,9%,17%,12%,16%,30%
71,2027,2,7N CARIBBEAN,16%,20%,11%,11%,24%,17%
72,2027,3,7N CARIBBEAN,28%,17%,18%,9%,14%,15%


In [13]:
# saves results
final.to_csv('my_wtd_report.csv', index=False)

In [14]:
# test example
# adjusted pax_build for each day
final[(final["Sail Year"] == 2025) & (final["Sail Month"] == 9) & (final["Product"] == "SHORT CARIBBEAN")]

,Sail Year,Sail Month,Product,Weekend,Monday,Tuesday,Wednesday,Thursday,Friday
30,2025,9,SHORT CARIBBEAN,23%,18%,14%,15%,19%,12%
